In [2]:
import numpy  as np

from scipy.stats import entropy
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.palettes import Inferno, all_palettes
from bokeh.models import CustomJS, Slider, ColumnDataSource 
from bokeh.models import Whisker, HoverTool, Span, ColorBar
from bokeh.transform import linear_cmap, log_cmap, factor_cmap

output_notebook()

Loading BokehJS ...

In [3]:
def plot_histogram(array):
    plot_options = dict(
                        title="d=4",
                        width=450,
                        plot_height=250,
                        x_range=(-0.6, 0.6),
                        tools='pan,wheel_zoom,reset,save')

    gate_hist = figure(**plot_options)

    hist, edges = np.histogram(array, bins=50)

    x = np.linspace(0, 1, 1000)

    gate_hist.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
         fill_color="#036564", line_color="#033649",\
    )

    
    gate_hist.yaxis.major_label_text_color = None 
    gate_hist.xaxis.axis_label = 'cosine similarity'
    gate_hist.yaxis.axis_label = 'Density'
#     gate_hist.title = ''
    show(gate_hist)

In [4]:
d = 4
n_samples = 1000

cos_sims = []

for i in range(n_samples):
    ## sampling from a hypercube
#     vec1 = np.random.uniform(-1, 1, size=(d))
#     vec2 = np.random.uniform(-1, 1, size=(d))
    
    ## sampling from a hypersphere
    u = np.random.normal(0,1,d)  # an array of d normally distributed random variables
    norm=np.sum(u**2) **(0.5)
    r = np.random.random()**(1.0/d)
    vec1 = r*u
    
    u = np.random.normal(0,1,d)  # an array of d normally distributed random variables
    norm=np.sum(u**2) **(0.5)
    r = np.random.random()**(1.0/d)
    vec2 = r*u
    
    cos_sims.append(np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))

plot_histogram(cos_sims)

## Step 1

In [9]:
def get_cos_memory(ref_memory, memory_bank):
    memory_dimension = memory_bank.shape[1]
    cosine_similarity_matrix = np.matmul(memory_bank, 
                                         ref_memory.reshape(memory_dimension, 1))
    
    cosine_similarity_matrix = cosine_similarity_matrix.flatten()/ \
                        (np.linalg.norm(memory_bank, axis=1).flatten()* \
                         np.linalg.norm(ref_memory)).flatten()
    
    ## Softmax computation
    attention_vector = np.exp(cosine_similarity_matrix)
    attention_vector = attention_vector/np.sum(attention_vector)

    final_memory = np.sum(attention_vector.reshape((-1, 1))*memory_bank, axis=0)
    
#     print(attention_vector.shape)
#     print(memory_bank.shape)
#     print(final_memory.shape)
    
    return final_memory.flatten()


def get_hop_memory(ref_memory, memory_bank, n_iters=2):
    in_memory = ref_memory.copy()
    in_memory = in_memory.reshape((-1, 1))
    for iter_idx in range(n_iters):
#         print(iter_idx)
        softmaxed = np.matmul(memory_bank, in_memory)
        softmaxed = np.exp(softmaxed)
        softmaxed = softmaxed/np.sum(softmaxed)
        out_memory = np.matmul(memory_bank.T, softmaxed)
        
        in_memory = out_memory
    
    return out_memory.flatten()


def run_memory_exp(memory_size=100, 
                   memory_dimension=4,
                   n_trials=1000,
                   noise_loc=0,
                   noise_spread=0,
                   hopfield_iterations=100):
    ## For simplicity assume vectors are in constrained to range (-1, 1)
    ## The strength which is the beta parameter is assumed to be 1 
    # Since the effect of the parameter is same for all the vectors, 
    # this assumption does not matter
    ## The noise model is gaussian

    c_deviations= []
    h_deviations = []
    c_entropy = []
    for i in range(n_trials):
        # Choose a memory to reference
        memory_bank = np.random.uniform(low=-1, high=1, size=(memory_size, memory_dimension))

        memory_choice = np.random.choice(range(memory_size))
        ref_memory = memory_bank[memory_choice, :].copy()

        # add noise to memory
        noise = np.random.normal(noise_loc, noise_spread, size=ref_memory.shape)
        ref_memory += noise

        # get the correspoding attention vectors
#         cos_memory = get_cos_memory(ref_memory, memory_bank)
        cos_memory = get_hop_memory(ref_memory, memory_bank, 1)
        hopfield_memory = get_hop_memory(ref_memory, memory_bank, hopfield_iterations)

        c_deviations.append(np.linalg.norm(np.abs(cos_memory-ref_memory)))
    #     print(ref_memory)
    #     print(cos_memory)
    #     print(hopfield_memory)
        h_deviations.append(np.linalg.norm(np.abs(hopfield_memory-ref_memory)))

    return c_deviations, h_deviations

In [10]:
memory_sizes = list(range(1, 1000, 50))
memory_dimensions = list(range(1, 100, 10))
# d = 50
# noise_spreads = [ 0, 0.001, 0.01, 0.05, 0.1 ]

plots = []
# for noise in noise_spreads:
cos_list = []
hopfield_list = []

for i in memory_dimensions:
    cosine_err, hopfield_err = run_memory_exp(memory_size=200, 
                                              memory_dimension=i,
                                              noise_loc=0,
                                              noise_spread=0,
                                              hopfield_iterations=5)
    cos_list.append(np.mean(cosine_err))
    hopfield_list.append(np.mean(hopfield_err))
    
#     plots.append(hopfield_list)

plot_options = dict(width=450,
                    plot_height=300,
                    tools='pan,wheel_zoom,reset,save')

comp_plot = figure(
                    title="s=200",
                   y_axis_type='log',
                   **plot_options)

comp_plot.line(memory_dimensions,
              cos_list,
              color='blue',
              legend_label="cosine similarity")

comp_plot.line(memory_dimensions,
              hopfield_list,
              color='red',
              legend_label="hopfield retrieval")

# palette = all_palettes['Inferno'][len(noise_spreads)]
# for idx, noise in enumerate(noise_spreads):
#     comp_plot.line(memory_dimensions,
#                   plots[idx],
#                   color=palette[idx],
#                   legend_label="noise std. dev={}".format(noise))

comp_plot.legend.location = 'bottom_left'
comp_plot.xaxis.axis_label = 'dimension of memory'
comp_plot.yaxis.axis_label = 'error in retrieval(log)'
show(comp_plot)